In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.float_format', '{:.2f}'.format)

## 1. Nhập dữ liệu vào notebook

In [2]:
# path to file
data_folder = "./Data"
os.makedirs(data_folder, exist_ok=True)
# Path to the "grouped_grouped.csv" file in the "Data" folder
file_full_path = os.path.join(data_folder, "raw_full_merged_df.csv")

# read file
raw_df = pd.read_csv(file_full_path)

In [3]:
raw_df.shape

(4287473, 14)

In [4]:
raw_df.sample()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
1928846,NaN,NaN,Jonkoping,Sweden,2020-06-20 04:33:19,57.37,14.34,2771.00,155.00,0.00,2616.00,"Jonkoping, Sweden",762.10,5.59


## 2. Drop các cột không dùng
(`Combined_Key`, `Incident_Rate`, `Case_Fatality_Ratio`, `Lat`, `Long_`)

In [5]:
drop_cols = ['Combined_Key','Incident_Rate','Case_Fatality_Ratio','Lat','Long_']
country_df = raw_df.copy()
country_df.drop(columns = drop_cols, inplace = True)
country_df.shape

(4287473, 9)

In [6]:
country_df.sample(10)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active
2454878,18169.00,Wabash,Indiana,US,2021-08-03 04:21:39,3731.00,84.00,NaN,NaN
479966,28121.00,Rankin,Mississippi,US,2023-02-10 04:20:54,46269.00,526.00,NaN,NaN
3315803,13125.00,Glascock,Georgia,US,2020-10-14 04:24:05,40.00,2.00,0.00,38.00
4083857,40045.00,Ellis,Oklahoma,US,2022-12-16 04:21:01,990.00,11.00,NaN,NaN
3532108,8063.00,Kit Carson,Colorado,US,2020-11-01 04:36:19,171.00,6.00,0.00,136.00
4080748,53067.00,Thurston,Washington,US,2021-12-16 04:22:11,23807.00,264.00,NaN,NaN
376185,39107.00,Mercer,Ohio,US,2021-02-02 05:22:49,4428.00,82.00,0.00,4346.00
3093095,39139.00,Richland,Ohio,US,2021-09-25 04:21:31,16602.00,272.00,NaN,NaN
959995,37007.00,Anson,North Carolina,US,2022-03-26 04:20:23,6542.00,96.00,NaN,NaN
3505401,29159.00,Pettis,Missouri,US,2022-10-29 04:21:13,14150.00,179.00,NaN,NaN


## 3. Drop các dòng không convert được:


In [7]:
values_to_drop = ['St. Martin', 'Cruise Ship', 'Diamond Princess', 'MS Zaandam', 'North Ireland',
                  'occupied Palestinian territory', 'Others', 'Summer Olympics 2020',
                  'Taipei and environs', 'Winter Olympics 2022','Azerbaijan','Antarctica', 'Holy See', 'Kiribati', 'Marshall Islands', 'Micronesia', 'Nauru', 'North Korea', 'Palau', 'Samoa', 'Solomon Islands', 'Tonga', 'Tuvalu', 'West Bank and Gaza']

# Loại bỏ các dòng có giá trị trong cột 'Country_Region' thuộc list values_to_drop
country_df = country_df[~country_df['Country_Region'].isin(values_to_drop)]
country_df.shape

(4269235, 9)

In [49]:
country_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active
0,NaN,NaN,NaN,Afghanistan,2021-01-02,52513.00,2201.00,41727.00,8585.00
1,NaN,NaN,NaN,Albania,2021-01-02,58316.00,1181.00,33634.00,23501.00
2,NaN,NaN,NaN,Algeria,2021-01-02,99897.00,2762.00,67395.00,29740.00
3,NaN,NaN,NaN,Andorra,2021-01-02,8117.00,84.00,7463.00,570.00
4,NaN,NaN,NaN,Angola,2021-01-02,17568.00,405.00,11146.00,6017.00


## 4. Xử lý date-time

In [8]:
# Tạo một bản sao của cột 'Last_Update' để giữ nguyên giá trị ban đầu (backup cho các dòng ko convert được)
country_df['Last_Update_original'] = country_df['Last_Update'].copy()

try:
    country_df['Last_Update'] = pd.to_datetime(country_df['Last_Update'], errors='coerce')
except:
    print("Can't convert!")

# Lọc và in ra các dòng không thể chuyển đổi
invalid_rows = country_df[country_df['Last_Update'].isna()]
invalid_rows

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Last_Update_original
156818,NaN,NaN,Andaman and Nicobar Islands,India,NaT,4976.00,62.00,4891.00,23.00,2021-01-15 17:22
156819,NaN,NaN,Andhra Pradesh,India,NaT,885616.00,7138.00,876140.00,2338.00,2021-01-15 17:22
156820,NaN,NaN,Arunachal Pradesh,India,NaT,16798.00,56.00,16674.00,68.00,2021-01-15 17:22
156821,NaN,NaN,Assam,India,NaT,216762.00,1065.00,212706.00,2991.00,2021-01-15 17:22
156822,NaN,NaN,Bihar,India,NaT,256895.00,1447.00,251278.00,4170.00,2021-01-15 17:22
...,...,...,...,...,...,...,...,...,...,...
4287468,NaN,NaN,Tennessee,US,NaT,371.00,1.00,0.00,NaN,3/21/2020 23:13
4287469,NaN,NaN,Wisconsin,US,NaT,282.00,4.00,0.00,NaN,3/21/2020 23:13
4287470,NaN,NaN,NaN,Cape Verde,NaT,1.00,0.00,0.00,NaN,3/21/2020 23:43
4287471,NaN,NaN,NaN,Papua New Guinea,NaT,1.00,0.00,0.00,NaN,3/21/2020 23:43


> Lọc ra các dòng có định dạng mm/dd/yy hh:mm

In [9]:
filtered_df = invalid_rows[invalid_rows['Last_Update_original'].str.contains(r'\d{1,2}/\d{1,2}/\d{2}\s\d{1,2}:\d{2}')]
filtered_df['Last_Update'] = pd.to_datetime(filtered_df['Last_Update_original'], format='%m/%d/%y %H:%M')
country_df.loc[filtered_df.index, 'Last_Update'] = filtered_df['Last_Update']

C:\Users\letra\AppData\Local\Temp\ipykernel_12580\2172368763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Last_Update'] = pd.to_datetime(filtered_df['Last_Update_original'], format='%m/%d/%y %H:%M')


Lọc lại các dòng không chuyển được

In [10]:
try:
    country_df['Last_Update'] = pd.to_datetime(country_df['Last_Update'], errors='coerce')
except:
    print("Can't convert!")

# Lọc và in ra các dòng không thể chuyển đổi
invalid_rows = country_df[country_df['Last_Update'].isna()]
invalid_rows


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Last_Update_original
156818,NaN,NaN,Andaman and Nicobar Islands,India,NaT,4976.00,62.00,4891.00,23.00,2021-01-15 17:22
156819,NaN,NaN,Andhra Pradesh,India,NaT,885616.00,7138.00,876140.00,2338.00,2021-01-15 17:22
156820,NaN,NaN,Arunachal Pradesh,India,NaT,16798.00,56.00,16674.00,68.00,2021-01-15 17:22
156821,NaN,NaN,Assam,India,NaT,216762.00,1065.00,212706.00,2991.00,2021-01-15 17:22
156822,NaN,NaN,Bihar,India,NaT,256895.00,1447.00,251278.00,4170.00,2021-01-15 17:22
...,...,...,...,...,...,...,...,...,...,...
4287468,NaN,NaN,Tennessee,US,NaT,371.00,1.00,0.00,NaN,3/21/2020 23:13
4287469,NaN,NaN,Wisconsin,US,NaT,282.00,4.00,0.00,NaN,3/21/2020 23:13
4287470,NaN,NaN,NaN,Cape Verde,NaT,1.00,0.00,0.00,NaN,3/21/2020 23:43
4287471,NaN,NaN,NaN,Papua New Guinea,NaT,1.00,0.00,0.00,NaN,3/21/2020 23:43


> Lọc ra các dòng có định dạng yyyy-mm-dd hh:mm

In [11]:
filtered_df = invalid_rows[invalid_rows['Last_Update_original'].apply(lambda x: len(str(x)) == 16)]
filtered_df['Last_Update'] = pd.to_datetime(filtered_df['Last_Update_original'], format='%Y-%m-%d %H:%M')
country_df.loc[filtered_df.index, 'Last_Update'] = filtered_df['Last_Update']


C:\Users\letra\AppData\Local\Temp\ipykernel_12580\2987739103.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Last_Update'] = pd.to_datetime(filtered_df['Last_Update_original'], format='%Y-%m-%d %H:%M')


Lọc lại các dòng không chuyển được

In [12]:
try:
    country_df['Last_Update'] = pd.to_datetime(country_df['Last_Update'], errors='coerce')
except:
    print("Can't convert!")

# Lọc và in ra các dòng không thể chuyển đổi
invalid_rows = country_df[country_df['Last_Update'].isna()]
invalid_rows


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Last_Update_original
4277676,NaN,NaN,Anhui,Mainland China,NaT,1.00,NaN,NaN,NaN,1/22/2020 17:00
4277677,NaN,NaN,Beijing,Mainland China,NaT,14.00,NaN,NaN,NaN,1/22/2020 17:00
4277678,NaN,NaN,Chongqing,Mainland China,NaT,6.00,NaN,NaN,NaN,1/22/2020 17:00
4277679,NaN,NaN,Cook Islands,New Zealand,NaT,0.00,0.00,0.00,NaN,1/22/2020 17:00
4277680,NaN,NaN,England,United Kingdom,NaT,0.00,0.00,0.00,NaN,1/22/2020 17:00
...,...,...,...,...,...,...,...,...,...,...
4287468,NaN,NaN,Tennessee,US,NaT,371.00,1.00,0.00,NaN,3/21/2020 23:13
4287469,NaN,NaN,Wisconsin,US,NaT,282.00,4.00,0.00,NaN,3/21/2020 23:13
4287470,NaN,NaN,NaN,Cape Verde,NaT,1.00,0.00,0.00,NaN,3/21/2020 23:43
4287471,NaN,NaN,NaN,Papua New Guinea,NaT,1.00,0.00,0.00,NaN,3/21/2020 23:43


Lọc ra các dòng có định dạng yyyy-mm-dd hh:mm

In [13]:
# filtered_df = invalid_rows[invalid_rows['Last_Update_original'].str.contains(r'\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}')]
# filtered_df['Last_Update'] = pd.to_datetime(filtered_df['Last_Update_original'], format='%Y-%m-%d %H:%M')
# country_df.loc[filtered_df.index, 'Last_Update'] = filtered_df['Last_Update']

Lọc lại các dòng không chuyển được

In [14]:
# try:
#     country_df['Last_Update'] = pd.to_datetime(country_df['Last_Update'], errors='coerce')
# except:
#     print("Can't convert!")

# # Lọc và in ra các dòng không thể chuyển đổi
# invalid_rows = country_df[country_df['Last_Update'].isna()]
# invalid_rows


Lọc ra các dòng có định dạng mm/dd/yyyy hh:mm

In [15]:
filtered_df = invalid_rows[invalid_rows['Last_Update_original'].str.contains(r'\d{1,2}/\d{1,2}/\d{4}\s\d{1,2}:\d{2}')]
filtered_df

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Last_Update_original
4277676,NaN,NaN,Anhui,Mainland China,NaT,1.00,NaN,NaN,NaN,1/22/2020 17:00
4277677,NaN,NaN,Beijing,Mainland China,NaT,14.00,NaN,NaN,NaN,1/22/2020 17:00
4277678,NaN,NaN,Chongqing,Mainland China,NaT,6.00,NaN,NaN,NaN,1/22/2020 17:00
4277679,NaN,NaN,Cook Islands,New Zealand,NaT,0.00,0.00,0.00,NaN,1/22/2020 17:00
4277680,NaN,NaN,England,United Kingdom,NaT,0.00,0.00,0.00,NaN,1/22/2020 17:00
...,...,...,...,...,...,...,...,...,...,...
4287468,NaN,NaN,Tennessee,US,NaT,371.00,1.00,0.00,NaN,3/21/2020 23:13
4287469,NaN,NaN,Wisconsin,US,NaT,282.00,4.00,0.00,NaN,3/21/2020 23:13
4287470,NaN,NaN,NaN,Cape Verde,NaT,1.00,0.00,0.00,NaN,3/21/2020 23:43
4287471,NaN,NaN,NaN,Papua New Guinea,NaT,1.00,0.00,0.00,NaN,3/21/2020 23:43


Chuyển sang datetime 

In [16]:
filtered_df['Last_Update'] = pd.to_datetime(filtered_df['Last_Update_original'], format='%m/%d/%Y %H:%M')
#filtered_df


Gán lại vào country_df

In [17]:
country_df.loc[filtered_df.index, 'Last_Update'] = filtered_df['Last_Update']


Lọc lại các dòng không chuyển được

In [18]:
try:
    country_df['Last_Update'] = pd.to_datetime(country_df['Last_Update'], errors='coerce')
except:
    print("Can't convert!")

# Lọc và in ra các dòng không thể chuyển đổi
invalid_rows = country_df[country_df['Last_Update'].isna()]
invalid_rows


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Last_Update_original


> Đã chuyển xong <3

In [19]:
country_df.dtypes

FIPS                           float64
Admin2                          object
Province_State                  object
Country_Region                  object
Last_Update             datetime64[ns]
Confirmed                      float64
Deaths                         float64
Recovered                      float64
Active                         float64
Last_Update_original            object
dtype: object

Xoá cột `Last_Update_original` và chuyển datetime về date

In [20]:
country_df.drop(columns=['Last_Update_original'], inplace=True)
country_df['Last_Update'] = pd.to_datetime(country_df['Last_Update']).dt.date
country_df

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active
0,NaN,NaN,NaN,Afghanistan,2021-01-02,52513.00,2201.00,41727.00,8585.00
1,NaN,NaN,NaN,Albania,2021-01-02,58316.00,1181.00,33634.00,23501.00
2,NaN,NaN,NaN,Algeria,2021-01-02,99897.00,2762.00,67395.00,29740.00
3,NaN,NaN,NaN,Andorra,2021-01-02,8117.00,84.00,7463.00,570.00
4,NaN,NaN,NaN,Angola,2021-01-02,17568.00,405.00,11146.00,6017.00
...,...,...,...,...,...,...,...,...,...
4287468,NaN,NaN,Tennessee,US,2020-03-21,371.00,1.00,0.00,NaN
4287469,NaN,NaN,Wisconsin,US,2020-03-21,282.00,4.00,0.00,NaN
4287470,NaN,NaN,NaN,Cape Verde,2020-03-21,1.00,0.00,0.00,NaN
4287471,NaN,NaN,NaN,Papua New Guinea,2020-03-21,1.00,0.00,0.00,NaN


In [21]:
country_df.dtypes

FIPS              float64
Admin2             object
Province_State     object
Country_Region     object
Last_Update        object
Confirmed         float64
Deaths            float64
Recovered         float64
Active            float64
dtype: object

In [22]:
country_df.shape

(4269235, 9)

In [23]:
# Chuyển đổi cột 'Last_Update' sang định dạng datetime
country_df['Last_Update'] = pd.to_datetime(country_df['Last_Update'])

# Lọc ra các dòng có Last_Update sau ngày 5/8/2021
drop_indices = country_df[country_df['Last_Update'] > '2021-08-05'].index

# Drop các dòng có index trong drop_indices
country_df = country_df.drop(drop_indices)

In [24]:
#fixing Country names
# ====================

# renaming countries, regions, provinces
country_df['Country_Region'] = country_df['Country_Region'].replace('Korea, South', 'South Korea')
country_df['Country_Region'] = country_df['Country_Region'].replace('Korea, North', 'North Korea')

# Greenland
country_df.loc[country_df['Province_State']=='Greenland', 'Country_Region'] = 'Greenland'

# Mainland china to China
country_df['Country_Region'] = country_df['Country_Region'].replace('Mainland China', 'China')

# Vietnam
country_df['Country_Region'] = country_df['Country_Region'].replace('Viet Nam', 'Vietnam')


In [25]:
# Active Case = confirmed - deaths - recovered
country_df['Active'] = country_df['Confirmed'] - country_df['Deaths'] - country_df['Recovered']
country_df.shape


(1947838, 9)

In [26]:
grouped_df = country_df.groupby(['Country_Region', 'Last_Update'])[['Confirmed','Deaths','Recovered','Active']].sum().reset_index()
grouped_df.shape

(94692, 6)

In [27]:
grouped_df

,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active
0,Azerbaijan,2020-02-28,1.00,0.00,0.00,1.00
1,Afghanistan,2020-02-24,68.00,0.00,0.00,68.00
2,Afghanistan,2020-03-08,16.00,0.00,0.00,16.00
3,Afghanistan,2020-03-10,8.00,0.00,0.00,8.00
4,Afghanistan,2020-03-11,33.00,0.00,0.00,33.00
...,...,...,...,...,...,...
94687,Zimbabwe,2021-08-01,108860.00,3532.00,75856.00,29472.00
94688,Zimbabwe,2021-08-02,109546.00,3583.00,76665.00,29298.00
94689,Zimbabwe,2021-08-03,110855.00,3635.00,79420.00,27800.00
94690,Zimbabwe,2021-08-04,112435.00,3676.00,81570.00,27189.00


# WHO Region

https://en.wikipedia.org/wiki/WHO_regions

In [28]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [29]:
# add 'WHO Region' column
grouped_df['WHO Region'] = grouped_df['Country_Region'].map(who_region)

# find missing values
grouped_df[grouped_df['WHO Region'].isna()]['Country_Region'].unique()

array([' Azerbaijan', 'Aruba', 'Bahamas, The', 'Cayman Islands',
       'Channel Islands', 'Curacao', 'East Timor', 'Faroe Islands',
       'French Guiana', 'Gambia, The', 'Gibraltar', 'Guadeloupe', 'Guam',
       'Guernsey', 'Hong Kong', 'Hong Kong SAR',
       'Iran (Islamic Republic of)', 'Jersey', 'Macao SAR', 'Macau',
       'Martinique', 'Mayotte', 'Puerto Rico', 'Republic of Ireland',
       'Republic of Korea', 'Republic of Moldova', 'Reunion',
       'Russian Federation', 'Saint Barthelemy', 'Saint Martin',
       'The Bahamas', 'The Gambia', 'Vatican City'], dtype=object)

In [30]:
grouped_df

,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,WHO Region
0,Azerbaijan,2020-02-28,1.00,0.00,0.00,1.00,NaN
1,Afghanistan,2020-02-24,68.00,0.00,0.00,68.00,Eastern Mediterranean
2,Afghanistan,2020-03-08,16.00,0.00,0.00,16.00,Eastern Mediterranean
3,Afghanistan,2020-03-10,8.00,0.00,0.00,8.00,Eastern Mediterranean
4,Afghanistan,2020-03-11,33.00,0.00,0.00,33.00,Eastern Mediterranean
...,...,...,...,...,...,...,...
94687,Zimbabwe,2021-08-01,108860.00,3532.00,75856.00,29472.00,Africa
94688,Zimbabwe,2021-08-02,109546.00,3583.00,76665.00,29298.00,Africa
94689,Zimbabwe,2021-08-03,110855.00,3635.00,79420.00,27800.00,Africa
94690,Zimbabwe,2021-08-04,112435.00,3676.00,81570.00,27189.00,Africa


## 5. Tính new case...

In [31]:
grouped_df['Last_Update'] = pd.to_datetime(grouped_df['Last_Update'])
# Tạo một cột mới cho 'Previous Day'
grouped_df['Previous Day'] = grouped_df.groupby(['Country_Region'])['Last_Update'].shift(1)
grouped_df

,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,WHO Region,Previous Day
0,Azerbaijan,2020-02-28,1.00,0.00,0.00,1.00,NaN,NaT
1,Afghanistan,2020-02-24,68.00,0.00,0.00,68.00,Eastern Mediterranean,NaT
2,Afghanistan,2020-03-08,16.00,0.00,0.00,16.00,Eastern Mediterranean,2020-02-24
3,Afghanistan,2020-03-10,8.00,0.00,0.00,8.00,Eastern Mediterranean,2020-03-08
4,Afghanistan,2020-03-11,33.00,0.00,0.00,33.00,Eastern Mediterranean,2020-03-10
...,...,...,...,...,...,...,...,...
94687,Zimbabwe,2021-08-01,108860.00,3532.00,75856.00,29472.00,Africa,2021-07-31
94688,Zimbabwe,2021-08-02,109546.00,3583.00,76665.00,29298.00,Africa,2021-08-01
94689,Zimbabwe,2021-08-03,110855.00,3635.00,79420.00,27800.00,Africa,2021-08-02
94690,Zimbabwe,2021-08-04,112435.00,3676.00,81570.00,27189.00,Africa,2021-08-03


In [32]:
# Tính toán New cases, New deaths và New recovered cho mỗi quốc gia
grouped_df['New cases'] = grouped_df.groupby('Country_Region')['Confirmed'].diff()
grouped_df['New deaths'] = grouped_df.groupby('Country_Region')['Deaths'].diff()
grouped_df['New recovered'] = grouped_df.groupby('Country_Region')['Recovered'].diff()

# Điền giá trị rỗng của hàng đầu tiên bằng giá trị của ngày hiện tại
grouped_df['New cases'].fillna(grouped_df['Confirmed'], inplace=True)
grouped_df['New deaths'].fillna(grouped_df['Deaths'], inplace=True)
grouped_df['New recovered'].fillna(grouped_df['Recovered'], inplace=True)

In [33]:
grouped_df

,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,WHO Region,Previous Day,New cases,New deaths,New recovered
0,Azerbaijan,2020-02-28,1.00,0.00,0.00,1.00,NaN,NaT,1.00,0.00,0.00
1,Afghanistan,2020-02-24,68.00,0.00,0.00,68.00,Eastern Mediterranean,NaT,68.00,0.00,0.00
2,Afghanistan,2020-03-08,16.00,0.00,0.00,16.00,Eastern Mediterranean,2020-02-24,-52.00,0.00,0.00
3,Afghanistan,2020-03-10,8.00,0.00,0.00,8.00,Eastern Mediterranean,2020-03-08,-8.00,0.00,0.00
4,Afghanistan,2020-03-11,33.00,0.00,0.00,33.00,Eastern Mediterranean,2020-03-10,25.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
94687,Zimbabwe,2021-08-01,108860.00,3532.00,75856.00,29472.00,Africa,2021-07-31,1370.00,42.00,1294.00
94688,Zimbabwe,2021-08-02,109546.00,3583.00,76665.00,29298.00,Africa,2021-08-01,686.00,51.00,809.00
94689,Zimbabwe,2021-08-03,110855.00,3635.00,79420.00,27800.00,Africa,2021-08-02,1309.00,52.00,2755.00
94690,Zimbabwe,2021-08-04,112435.00,3676.00,81570.00,27189.00,Africa,2021-08-03,1580.00,41.00,2150.00


In [34]:
grouped_df.describe()

,Last_Update,Confirmed,Deaths,Recovered,Active,Previous Day,New cases,New deaths,New recovered
count,94692,94692.00,94692.00,94692.00,94692.00,94468,94692.00,94692.00,94692.00
mean,2020-11-21 19:08:54.228868608,406973.17,9712.42,246794.55,99317.85,2020-11-21 09:37:41.337172736,2113.28,45.14,1375.57
min,2020-01-22 00:00:00,0.00,0.00,0.00,-163828.00,2020-01-22 00:00:00,-21887422.00,-200626.00,-21360928.00
25%,2020-07-17 00:00:00,1632.00,27.00,721.00,191.00,2020-07-17 00:00:00,3.00,0.00,0.00
50%,2020-11-23 00:00:00,14693.50,260.00,8857.00,2091.00,2020-11-22 00:00:00,74.00,1.00,23.00
75%,2021-03-31 00:00:00,141729.75,2728.00,92667.00,17663.25,2021-03-30 00:00:00,727.00,11.00,419.00
max,2021-08-05 00:00:00,35909045.00,632648.00,30974748.00,28612316.00,2021-08-04 00:00:00,21920743.00,200902.00,21392193.00
std,NaN,2072461.68,41261.85,1265986.08,829607.88,NaN,103658.74,1039.98,100975.26


Kiểm tra các giá trị âm và fix

In [35]:
neg_cases = grouped_df[grouped_df['New cases'] < 0]
neg_deaths = grouped_df[grouped_df['New deaths'] < 0]
neg_recovered = grouped_df[grouped_df['New recovered'] < 0]
print(neg_cases.shape)
print(neg_deaths.shape)
print(neg_recovered.shape)

(499, 11)
(193, 11)
(352, 11)


Lặp 6 lần để fix - update - fix 

In [36]:
i=0
while (i<6):
    # CASES
    for index, row in neg_cases.iterrows():
        # Lấy dòng từ grouped_df dựa trên "Country_Region", "WHO Region", và "Last_Update" là ngày liền trước
        mask = (grouped_df['Country_Region'] == row['Country_Region']) & \
            (grouped_df['WHO Region'] == row['WHO Region']) & \
            (grouped_df['Last_Update'] == row['Previous Day'])
        grouped_df.loc[mask, 'Confirmed'] = grouped_df.loc[mask, 'Confirmed'] + row['New cases']

        mask2 = (grouped_df['Country_Region'] == row['Country_Region']) & \
            (grouped_df['WHO Region'] == row['WHO Region']) & \
            (grouped_df['Last_Update'] == row['Last_Update'])
        grouped_df.loc[mask2, 'New cases']= row['Confirmed'] - grouped_df.loc[mask, 'Confirmed']
        grouped_df['New cases'] = grouped_df.groupby('Country_Region')['Confirmed'].diff()
        grouped_df['New cases'].fillna(grouped_df['Confirmed'], inplace=True)
    neg_cases = grouped_df[grouped_df['New cases'] < 0]

    # DEATHS
    for index, row in neg_deaths.iterrows():
        # Lấy dòng từ grouped_df dựa trên "Country_Region", "WHO Region", và "Last_Update" là ngày liền trước
        mask = (grouped_df['Country_Region'] == row['Country_Region']) & \
            (grouped_df['WHO Region'] == row['WHO Region']) & \
            (grouped_df['Last_Update'] == row['Previous Day'])
        grouped_df.loc[mask, 'Deaths'] = grouped_df.loc[mask, 'Deaths'] + row['New deaths']

        mask2 = (grouped_df['Country_Region'] == row['Country_Region']) & \
            (grouped_df['WHO Region'] == row['WHO Region']) & \
            (grouped_df['Last_Update'] == row['Last_Update'])
        grouped_df.loc[mask2, 'New deaths']=row['Deaths'] - grouped_df.loc[mask, 'Deaths']
        grouped_df['New deaths'] = grouped_df.groupby('Country_Region')['Deaths'].diff()
        grouped_df['New deaths'].fillna(grouped_df['Deaths'], inplace=True)
    neg_deaths = grouped_df[grouped_df['New deaths'] < 0]

    # RECOVERED
    for index, row in neg_recovered.iterrows():
        # Lấy dòng từ grouped_df dựa trên "Country_Region", "WHO Region", và "Last_Update" là ngày liền trước
        mask = (grouped_df['Country_Region'] == row['Country_Region']) & \
            (grouped_df['WHO Region'] == row['WHO Region']) & \
            (grouped_df['Last_Update'] == row['Previous Day'])
        grouped_df.loc[mask, 'Recovered'] = grouped_df.loc[mask, 'Recovered'] + row['New recovered']

        mask2 = (grouped_df['Country_Region'] == row['Country_Region']) & \
            (grouped_df['WHO Region'] == row['WHO Region']) & \
            (grouped_df['Last_Update'] == row['Last_Update'])
        grouped_df.loc[mask2, 'New recovered']= row['Recovered'] - grouped_df.loc[mask, 'Recovered']
        grouped_df['New recovered'] = grouped_df.groupby('Country_Region')['Recovered'].diff()
        grouped_df['New recovered'].fillna(grouped_df['Recovered'], inplace=True)
    neg_recovered = grouped_df[grouped_df['New recovered'] < 0]

    i+=1


In [37]:
neg_cases = grouped_df[grouped_df['New cases'] < 0]
neg_deaths = grouped_df[grouped_df['New deaths'] < 0]
neg_recovered = grouped_df[grouped_df['New recovered'] < 0]
print(neg_cases.shape)
print(neg_deaths.shape)
print(neg_recovered.shape)

(49, 11)
(22, 11)
(42, 11)


Số dòng âm rất nhỏ so với số dòng dữ liệu => drop

In [38]:
grouped_df.drop(neg_cases.index, inplace = True)
neg_deaths = grouped_df[grouped_df['New deaths'] < 0]
grouped_df.drop(neg_deaths.index, inplace = True)
neg_recovered = grouped_df[grouped_df['New recovered'] < 0]
grouped_df.drop(neg_recovered.index, inplace = True)


In [39]:
# Active Case = confirmed - deaths - recovered
grouped_df['Active'] = grouped_df['Confirmed'] - grouped_df['Deaths'] - grouped_df['Recovered']

In [40]:
grouped_df.loc[(grouped_df['Active'] < 0),'Active']=grouped_df.loc[(grouped_df['Active'] < 0),'New cases']
grouped_df.drop(columns= "Previous Day", inplace = True)
grouped_df['Last_Update'] = pd.to_datetime(grouped_df['Last_Update']).dt.date


In [41]:
grouped_df.describe()

,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
count,94598.00,94598.00,94598.00,94598.00,94598.00,94598.00,94598.00
mean,405022.71,9676.64,246273.08,149302.89,2158.14,46.17,1449.80
std,2055862.38,41029.80,1262880.51,1506045.42,14634.72,297.73,10117.33
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,1633.00,27.00,721.00,201.00,2.00,0.00,0.00
50%,14677.00,260.00,8846.50,2151.00,73.00,1.00,22.00
75%,141704.25,2718.00,92610.50,17971.25,721.00,11.00,416.00
max,35460627.00,611219.00,30974748.00,34849408.00,2349350.00,45576.00,1123456.00


# Country wise latest

In [42]:
# Country wise
# ============
full_grouped = grouped_df.copy()

full_grouped['Last_Update'] = pd.to_datetime(full_grouped['Last_Update'])

# getting latest values
country_wise = full_grouped[full_grouped['Last_Update']==max(full_grouped['Last_Update'])] \
                    .reset_index(drop=True) \
                    .drop('Last_Update', axis=1)

print(country_wise.shape)

# group by country
country_wise = country_wise.groupby('Country_Region')[['Confirmed', 'Deaths',
                                                      'Recovered', 'Active',
                                                      'New cases', 'New deaths', 'New recovered']].sum().reset_index()
print(country_wise.shape)


# per 100 cases
country_wise['Deaths / 100 Cases'] = round((country_wise['Deaths']/country_wise['Confirmed'])*100, 2)
country_wise['Recovered / 100 Cases'] = round((country_wise['Recovered']/country_wise['Confirmed'])*100, 2)
country_wise['Deaths / 100 Recovered'] = round((country_wise['Deaths']/country_wise['Recovered'])*100, 2)

cols = ['Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered']
country_wise[cols] = country_wise[cols].fillna(0)


# 1 week increase and % change
# ============================

today = full_grouped[full_grouped['Last_Update']==max(full_grouped['Last_Update'])] \
            .reset_index(drop=True) \
            .drop('Last_Update', axis=1)[['Country_Region', 'Confirmed']]

last_week = full_grouped[full_grouped['Last_Update']==max(full_grouped['Last_Update'])-timedelta(days=7)] \
                .reset_index(drop=True) \
                .drop('Last_Update', axis=1)[['Country_Region', 'Confirmed']]

temp = pd.merge(today, last_week, on='Country_Region', suffixes=(' today', ' last week'))
temp['1 week change'] = temp['Confirmed today'] - temp['Confirmed last week']
temp = temp[['Country_Region', 'Confirmed last week', '1 week change']]

country_wise = pd.merge(country_wise, temp, on='Country_Region')
country_wise['1 week % increase'] = round(country_wise['1 week change']/country_wise['Confirmed last week']*100, 2)
country_wise.head()

country_wise['WHO Region'] = country_wise['Country_Region'].map(who_region)
country_wise[country_wise['WHO Region'].isna()]['Country_Region'].unique()

country_wise.head()

(185, 9)
(185, 8)


,Country_Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,148933.00,6836.00,82586.00,59511.00,361.00,32.00,0.00,4.59,55.45,8.28,145552.00,3381.00,2.32,Eastern Mediterranean
1,Albania,133310.00,2457.00,130314.00,539.00,99.00,0.00,23.00,1.84,97.75,1.89,132952.00,358.00,0.27,Europe
2,Algeria,176724.00,4404.00,118409.00,53911.00,1495.00,34.00,852.00,2.49,67.00,3.72,167131.00,9593.00,5.74,Africa
3,Andorra,14797.00,128.00,14380.00,289.00,31.00,0.00,32.00,0.87,97.18,0.89,14586.00,211.00,1.45,Europe
4,Angola,43158.00,1026.00,39582.00,2550.00,88.00,4.00,193.00,2.38,91.71,2.59,42288.00,870.00,2.06,Africa


In [43]:
country_pop = {'Afghanistan':39009447,
'Albania':2877470,
'Algeria':43926079,
'Andorra':77278,
'Angola':32956300,
'Antigua and Barbuda':98010,
'Argentina':45236884,
'Armenia':2963811,
'Australia':25528864,
'Austria':9011577,
'Azerbaijan':10148243,
'Bahamas':393616,
'Bahrain':1706669,
'Bangladesh':164851401,
'Barbados':287411,
'Belarus':9449001,
'Belgium':11594739,
'Belize':398312,
'Benin':12151976,
'Bhutan':772443,
'Bolivia':11688459,
'Bosnia and Herzegovina':3278650,
'Botswana':2356075,
'Brazil':212710692,
'Brunei':437893,
'Bulgaria':6942854,
'Burkina Faso':20954852,
'Burma':54446389,
'Burundi':11922216,
'Cabo Verde':556581,
'Cambodia':16741375,
'Cameroon':26606188,
'Canada':37775022,
'Central African Republic':4837752,
'Chad':16467965,
'Chile':19132514,
'China':1425887337,
'Colombia':50936262,
'Comoros':871326,
'Congo (Brazzaville)':5530506,
'Congo (Kinshasa)':5530506,
'Costa Rica':5098730,
"Cote d'Ivoire":28160542,
'Croatia':4102577,
'Cuba':11325899,
'Cyprus':1208238,
'Czechia':10711019,
'Denmark':5794279,
'Djibouti':989387,
'Dominica':72004,
'Dominican Republic':10858648,
'Ecuador':17668824,
'Egypt':102516525,
'El Salvador':6489514,
'Equatorial Guinea':1407001,
'Eritrea':3551175,
'Estonia':1326627,
'Eswatini':1161348,
'Ethiopia':115223736,
'Fiji':897095,
'Finland':5541604,
'France':65288306,
'Gabon':2230563,
'Gambia':2422754,
'Georgia':3988368,
'Germany':83811260,
'Ghana':31133483,
'Greece':10417673,
'Greenland':56780,
'Grenada':112576,
'Guatemala':17946899,
'Guinea':13164905,
'Guinea-Bissau':1972277,
'Guyana':786936,
'Haiti':11416103,
'Honduras':9919704,
'Hungary':9657785,
'Iceland':341465,
'India':1381344997,
'Indonesia':273808365,
'Iran':84097623,
'Iraq':40306025,
'Ireland':4943200,
'Israel':9197590,
'Italy':60452568,
'Jamaica':2962478,
'Japan':126435859,
'Jordan':10213138,
'Kazakhstan':18798667,
'Kenya':53881160,
'Kosovo':1771315,
'Kuwait':4276658,
'Kyrgyzstan':6534479,
'Laos':7285750,
'Latvia':1883936,
'Lebanon':6822220,
'Lesotho':2143943,
'Liberia':5068618,
'Libya':6880353,
'Liechtenstein':38139,
'Lithuania':2718121,
'Luxembourg':626952,
'Madagascar':27755708,
'Malawi':19174839,
'Malaysia':32406372,
'Maldives':541448,
'Mali':20302901,
'Malta':441663,
'Mauritania':4660728,
'Mauritius':1271985,
'Mexico':129066160,
'Moldova':4032983,
'Monaco':39270,
'Mongolia':3283344,
'Montenegro':628074,
'Morocco':36953359,
'Mozambique':31333962,
'Namibia':2545264,
'Nepal':29186486,
'Netherlands':17138756,
'New Zealand':5002100,
'Nicaragua':6632263,
'Niger':24281433,
'Nigeria':206606300,
'North Macedonia':2083365,
'Norway':5425471,
'Oman':5118446,
'Pakistan':221295851,
'Panama':4321282,
'Papua New Guinea':8963009,
'Paraguay':7141091,
'Peru':33016319,
'Philippines':109722719,
'Poland':37842302,
'Portugal':10193593,
'Qatar':2807805,
'Romania':19224023,
'Russia':145940924,
'Rwanda':12981546,
'Saint Kitts and Nevis':53237,
'Saint Lucia':183712,
'Saint Vincent and the Grenadines':110976,
'San Marino':33938,
'Sao Tome and Principe':219544,
'Saudi Arabia':34865919,
'Senegal':16783877,
'Serbia':8733665,
'Seychelles':98408,
'Sierra Leone':7992169,
'Singapore':5854932,
'Slovakia':5459915,
'Slovenia':2078968,
'Somalia':15933012,
'South Africa':59381566,
'South Korea':51273732,
'South Sudan':11206572,
'Spain':46756648,
'Sri Lanka':21422362,
'Sudan':43943536,
'Suriname':587154,
'Sweden':10105596,
'Switzerland':8660952,
'Syria':17539600,
'Taiwan*':23821199,
'Tajikistan':9557468,
'Tanzania':59886383,
'Thailand':69817894,
'Timor-Leste':1320812,
'Togo':8296582,
'Trinidad and Tobago':1399950,
'Tunisia':11830801,
'Turkey':84428331,
'US':331198130,
'Uganda':45867852,
'Ukraine':43705858,
'United Arab Emirates':9902079,
'United Kingdom':67922029,
'Uruguay':3474956,
'Uzbekistan':33516027,
'Vanuatu':326740,
'Venezuela':28427499,
'Vietnam':97425470,
'Yemen':29886897,
'Zambia':18430129,
'Zimbabwe':14883803
}

In [44]:
country_wise['Population'] = country_wise['Country_Region'].map(country_pop)
country_wise

,Country_Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region,Population
0,Afghanistan,148933.00,6836.00,82586.00,59511.00,361.00,32.00,0.00,4.59,55.45,8.28,145552.00,3381.00,2.32,Eastern Mediterranean,39009447.00
1,Albania,133310.00,2457.00,130314.00,539.00,99.00,0.00,23.00,1.84,97.75,1.89,132952.00,358.00,0.27,Europe,2877470.00
2,Algeria,176724.00,4404.00,118409.00,53911.00,1495.00,34.00,852.00,2.49,67.00,3.72,167131.00,9593.00,5.74,Africa,43926079.00
3,Andorra,14797.00,128.00,14380.00,289.00,31.00,0.00,32.00,0.87,97.18,0.89,14586.00,211.00,1.45,Europe,77278.00
4,Angola,43158.00,1026.00,39582.00,2550.00,88.00,4.00,193.00,2.38,91.71,2.59,42288.00,870.00,2.06,Africa,32956300.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,Venezuela,309218.00,3649.00,294607.00,10962.00,766.00,12.00,935.00,1.18,95.27,1.24,302988.00,6230.00,2.06,Americas,28427499.00
181,Vietnam,181756.00,2327.00,54332.00,125097.00,7295.00,256.00,3501.00,1.28,29.89,4.28,123640.00,58116.00,47.00,Western Pacific,97425470.00
182,Yemen,7096.00,1380.00,4251.00,1465.00,10.00,0.00,19.00,19.45,59.91,32.46,7027.00,69.00,0.98,Eastern Mediterranean,29886897.00
183,Zambia,197791.00,3430.00,189658.00,4703.00,668.00,8.00,317.00,1.73,95.89,1.81,193432.00,4359.00,2.25,Africa,18430129.00


In [45]:
# save as .csv file
country_wise.to_csv('./Data/country_wise_latest_2021.csv', index=False)

In [46]:
grouped_df['Population'] = grouped_df['Country_Region'].map(country_pop)
grouped_df

,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,WHO Region,New cases,New deaths,New recovered,Population
0,Azerbaijan,2020-02-28,1.00,0.00,0.00,1.00,NaN,1.00,0.00,0.00,NaN
1,Afghanistan,2020-02-24,8.00,0.00,0.00,8.00,Eastern Mediterranean,8.00,0.00,0.00,39009447.00
2,Afghanistan,2020-03-08,8.00,0.00,0.00,8.00,Eastern Mediterranean,0.00,0.00,0.00,39009447.00
3,Afghanistan,2020-03-10,8.00,0.00,0.00,8.00,Eastern Mediterranean,0.00,0.00,0.00,39009447.00
4,Afghanistan,2020-03-11,14.00,0.00,0.00,14.00,Eastern Mediterranean,6.00,0.00,0.00,39009447.00
...,...,...,...,...,...,...,...,...,...,...,...
94687,Zimbabwe,2021-08-01,108860.00,3532.00,75856.00,29472.00,Africa,1370.00,42.00,1294.00,14883803.00
94688,Zimbabwe,2021-08-02,109546.00,3583.00,76665.00,29298.00,Africa,686.00,51.00,809.00,14883803.00
94689,Zimbabwe,2021-08-03,110855.00,3635.00,79420.00,27800.00,Africa,1309.00,52.00,2755.00,14883803.00
94690,Zimbabwe,2021-08-04,112435.00,3676.00,81570.00,27189.00,Africa,1580.00,41.00,2150.00,14883803.00


Xuất fulldata

In [47]:
grouped_df.to_csv('./Data/full_data_final.csv', index = False)

In [48]:
grouped_df.shape

(94598, 11)